<a href="https://colab.research.google.com/github/hwangho-kim/Utility-OAC/blob/main/CSV_%ED%8C%8C%EC%9D%BC_%EC%B9%BC%EB%9F%BC_%ED%8F%89%EA%B7%A0_%EB%B0%8F_%ED%95%A9%EA%B3%84_%EA%B3%84%EC%82%B0%2C_%EC%A0%80%EC%9E%A5_%EC%8A%A4%ED%81%AC%EB%A6%BD%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import re
from collections import defaultdict

def process_csv_files_with_aggregation(input_folder, output_folder):
    """
    지정된 폴더의 CSV 파일들을 읽어 특정 패턴의 칼럼들의 평균 또는 합계를 계산하고,
    결과를 새로운 CSV 파일로 저장합니다.
    '전력'이 포함된 칼럼 그룹은 합계를 계산하고 _SUM 접미사를 붙입니다.

    Args:
        input_folder (str): 입력 CSV 파일들이 있는 폴더 경로.
        output_folder (str): 처리된 CSV 파일들을 저장할 폴더 경로.
    """
    # 출력 폴더가 존재하지 않으면 생성합니다.
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"출력 폴더 '{output_folder}'를 생성했습니다.")

    # 집계 대상 칼럼을 식별하기 위한 정규 표현식입니다.
    # 패턴: basename_X 또는 basename_X_Y (여기서 X는 1, 2, 또는 3이고, Y는 숫자입니다)
    # 예: data_1, feature_2_10, metric_3_5
    column_pattern = re.compile(r"^(.*?)_([1-3])(_\d+)?$")

    # 입력 폴더 내의 모든 파일을 확인합니다.
    for filename in os.listdir(input_folder):
        if filename.endswith(".csv"):
            input_file_path = os.path.join(input_folder, filename)
            # 출력 파일 이름은 원본 파일 이름에 "_AGGREGATED" (또는 유사한 이름)을 추가할 수 있으나,
            # 여기서는 기존 "_AVG"를 유지하고, 실제 연산에 따라 _SUM 또는 _AVG가 칼럼명에 붙도록 합니다.
            # 파일명 자체는 "_PROCESSED" 등으로 변경하는 것이 더 명확할 수 있습니다.
            # 일단은 이전 요청대로 "_AVG" 파일명 접미사를 유지하되, 칼럼명은 조건에 따라 달라집니다.
            # 사용자의 요청은 _AVG 칼럼으로 저장하고, '전력' 포함시 _SUM으로 저장하므로
            # 파일명은 _AVG.csv로 유지하고 내부 칼럼명만 변경합니다.
            output_file_name = f"{os.path.splitext(filename)[0]}_PROCESSED.csv" # 파일명 변경 제안
            # output_file_name = f"{os.path.splitext(filename)[0]}_AVG.csv" # 이전 요청 유지 시
            output_file_path = os.path.join(output_folder, output_file_name)

            print(f"파일 '{filename}' 처리 중...")

            try:
                # CSV 파일을 읽어들입니다.
                df = pd.read_csv(input_file_path)

                if df.empty:
                    print(f"  파일 '{filename}'이 비어있습니다. 원본 내용 그대로 '{output_file_name}'으로 저장합니다.")
                    df.to_csv(output_file_path, index=False, encoding='utf-8-sig')
                    continue

                # 집계할 칼럼들을 그룹별로 저장할 딕셔너리입니다.
                cols_to_aggregate_map = defaultdict(list)
                # 집계에 사용되지 않고 그대로 유지될 칼럼들의 리스트입니다.
                original_cols_to_keep = []

                # 데이터프레임의 모든 칼럼을 순회합니다.
                for col in df.columns:
                    match = column_pattern.match(col)
                    if match:
                        # 정규 표현식에 매칭되면, 베이스 이름(접두사)을 추출합니다.
                        base_name = match.group(1)
                        cols_to_aggregate_map[base_name].append(col)
                    else:
                        # 매칭되지 않으면 원본 유지 칼럼 리스트에 추가합니다.
                        original_cols_to_keep.append(col)

                # 최종 결과 데이터프레임을 구성할 파트들을 담을 리스트입니다.
                # 먼저, 집계에 사용되지 않는 원본 칼럼들을 추가합니다.
                result_df_parts = [df[original_cols_to_keep].copy()] # .copy() to avoid SettingWithCopyWarning

                # 그룹화된 칼럼들의 평균 또는 합계를 계산하고 결과 파트에 추가합니다.
                if not cols_to_aggregate_map:
                    print(f"  파일 '{filename}'에서 집계 대상 칼럼을 찾지 못했습니다.")

                for base_name, related_cols in cols_to_aggregate_map.items():
                    if related_cols:
                        # 집계 전에 해당 칼럼들을 숫자형으로 변환 시도합니다.
                        # 숫자로 변환할 수 없는 값은 NaN으로 처리됩니다.
                        numeric_cols_df = df[related_cols].apply(pd.to_numeric, errors='coerce')

                        agg_col_name = ""
                        agg_series = None

                        # 칼럼 기본 이름에 '전력'이 포함되어 있는지 확인합니다.
                        if "전력" in base_name:
                            agg_col_name = f"{base_name}_SUM"
                            # row별 합계를 계산합니다 (NaN 값은 0으로 처리 후 합산 또는 skipna=True 기본값 사용).
                            agg_series = numeric_cols_df.sum(axis=1, skipna=True)
                            print(f"  칼럼 그룹 '{base_name}' (칼럼: {related_cols})의 합계를 '{agg_col_name}'으로 계산했습니다.")
                        else:
                            agg_col_name = f"{base_name}_AVG"
                            # row별 평균을 계산합니다 (NaN 값은 무시).
                            agg_series = numeric_cols_df.mean(axis=1, skipna=True)
                            print(f"  칼럼 그룹 '{base_name}' (칼럼: {related_cols})의 평균을 '{agg_col_name}'으로 계산했습니다.")

                        if agg_series is not None:
                            agg_series.name = agg_col_name
                            result_df_parts.append(agg_series)

                # 모든 파트를 하나로 합쳐 최종 데이터프레임을 만듭니다.
                if not result_df_parts:
                    final_df = pd.DataFrame()
                else:
                    final_df = pd.concat(result_df_parts, axis=1)

                # 처리된 데이터프레임을 CSV 파일로 저장합니다.
                # 한글 깨짐 방지를 위해 'utf-8-sig' 인코딩을 사용합니다.
                final_df.to_csv(output_file_path, index=False, encoding='utf-8-sig')
                print(f"  파일 '{filename}' 처리 완료. 저장 위치: '{output_file_path}'")

            except pd.errors.EmptyDataError:
                print(f"  파일 '{filename}'이 비어있거나 유효한 CSV 형식이 아닙니다. 건너<0xEB><0><0xB5>니다.") # "건너<0xEB><0><0xB5>니다" -> "건너뜁니다" 수정 필요
            except Exception as e:
                print(f"  파일 '{filename}' 처리 중 오류 발생: {e}")
    print("\n모든 CSV 파일 처리가 완료되었습니다.")

if __name__ == '__main__':
    # 사용 예시:
    # 아래 'your_input_csv_folder_path'와 'your_output_csv_folder_path'를
    # 실제 CSV 파일들이 있는 폴더 경로와 결과를 저장할 폴더 경로로 변경해주세요.

    # 예: input_directory = r"C:\Users\YourUser\Desktop\InputCSVs"
    # 예: output_directory = r"C:\Users\YourUser\Desktop\OutputCSVs"

    input_directory = "your_input_csv_folder_path"  # << 여기에 입력 CSV 폴더 경로를 입력하세요.
    output_directory = "your_output_csv_folder_path" # << 여기에 출력 CSV 폴더 경로를 입력하세요.

    # 아래 print문의 오타 수정: "건너<0xEB><0><0xB5>니다." -> "건너뜁니다."
    # 이 부분은 실제 코드 실행 로직이 아니라, 사용자에게 안내하는 부분입니다.
    # 실제 코드 내의 print문은 위에서 수정되었습니다.

    if input_directory == "your_input_csv_folder_path" or \
       output_directory == "your_output_csv_folder_path":
        print("스크립트 하단의 'input_directory'와 'output_directory' 변수에\n"
              "실제 폴더 경로를 입력한 후 다시 실행해주세요.")
    else:
        # 함수 이름 변경: process_csv_files_with_averaging -> process_csv_files_with_aggregation
        process_csv_files_with_aggregation(input_directory, output_directory)